In [1]:
import evaluate
import tokenizers
import numpy as np
import pandas as pd
import transformers
from datetime import datetime
from datasets import load_dataset, load_from_disk, concatenate_datasets
from transformers import (AutoTokenizer, DataCollatorForLanguageModeling, AlbertForMaskedLM, AutoConfig, TrainingArguments, 
                          Trainer, AutoModelForSequenceClassification)

In [2]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [3]:
def tokenize_function2(examples):
    return my_tokenizer(examples['text'], padding="max_length", truncation=True)

In [4]:
dataset = load_from_disk('../CLIdata/datasets/cuneiform-spaced-indexed')
my_tokenizer = AutoTokenizer.from_pretrained('../tokenizers/bert-base-uncased_train_val_test_maxlen_512_vocab_size_1000')
vocab_size = my_tokenizer.vocab_size
max_len = my_tokenizer.model_max_length

In [5]:
metric = evaluate.load('accuracy')

In [6]:
path = '../checkpoints-albert/albert_batch16_lr1e-5_pre_train_val_test/checkpoint-860000/'
classifier = AutoModelForSequenceClassification.from_pretrained(path, num_labels=7)

Some weights of the model checkpoint at ../checkpoints-albert/albert_batch16_lr1e-5_pre_train_val_test/checkpoint-860000/ were not used when initializing AlbertForSequenceClassification: ['predictions.LayerNorm.bias', 'predictions.dense.bias', 'predictions.decoder.weight', 'predictions.LayerNorm.weight', 'predictions.bias', 'predictions.dense.weight', 'predictions.decoder.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at ../checkpoints-albert/a

In [7]:
tokenized_dataset2 = dataset.map(tokenize_function2, batched=True)

Loading cached processed dataset at ../CLIdata/datasets/cuneiform-spaced-indexed/train\cache-419e017472122815.arrow
Loading cached processed dataset at ../CLIdata/datasets/cuneiform-spaced-indexed/val\cache-bbeda502a553fac7.arrow


  0%|          | 0/7 [00:00<?, ?ba/s]

In [8]:
dia = datetime.today().strftime("%Y-%M-%d")
hora = datetime.now().strftime("%H-%M")

batch = 32
lr = 1e-5
training_args2 = TrainingArguments(
    output_dir=f'../checkpoints_albert_finetuning/batch{batch}_lr{lr}_{dia}_hora_{hora}',
    overwrite_output_dir=True,
    per_device_train_batch_size=batch,
    learning_rate=lr,
    logging_strategy='steps',
    logging_steps=1e3,
    save_strategy='steps',
    save_steps=10_000, 
    max_steps= 100_000, #200_000
    evaluation_strategy="epoch"
)

trainer2 = Trainer(
    model=classifier,
    args=training_args2,
    #data_collator=data_collator_padding, #o default data_collator ja era o DataCollatorWithPadding, entao nao era neceessario ter criado um data_collator anteriomente
    train_dataset=concatenate_datasets([tokenized_dataset2['train'],tokenized_dataset2['val']]),
    eval_dataset=tokenized_dataset2['test'],
    compute_metrics=compute_metrics
)

max_steps is given, it will override any value given in num_train_epochs


In [9]:
trainer2.train()

The following columns in the training set don't have a corresponding argument in `AlbertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `AlbertForSequenceClassification.forward`,  you can safely ignore this message.
c:\Users\igorr\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 91130
  Num Epochs = 36
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 100000
  Number of trainable parameters = 11688967


  0%|          | 0/100000 [00:00<?, ?it/s]